In [1]:
"""
After webscrapping, we have 9 csvs, each representing job posting results of a job search in Singapore. This notebook, merges these 8 
csvs and does data cleaning.
"""

"""
If include japan, need edit:
1. uploading of datasets and concatenation
2. simplify_location
"""

'\nIf include japan, need edit:\n1. uploading of datasets and concatenation\n2. simplify_location\n'

In [2]:
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

/Users/joshuakoh/miniconda3/envs/gd_v3/lib/python3.11/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/joshuakoh/miniconda3/envs/gd_v3/lib/python3.11/site-packages/visions/backends/shared/nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit


In [3]:
csv_filenames = ["Business_Analyst_extracted_data.csv",
                 "Data_Analyst_extracted_data.csv",
                 "Data_Architect_extracted_data.csv",
                "Data_Engineer_extracted_data.csv",
                "Data_Scientist_extracted_data.csv",
                "Database_Administrator_extracted_data.csv",
                "Machine_Learning_Engineer_extracted_data.csv",
                "Software_Engineer_extracted_data.csv",
                "statistician_extracted_data.csv"]
datasets = [pd.read_csv(i) for i in csv_filenames]
df = pd.concat(datasets)

In [4]:
df = df.drop('Unnamed: 0',axis = 1) #Remove the unnamed column
df = df.drop_duplicates(subset = "job_id") #Remove duplicates (base on job id)
df.head()

,job_id,job_name,job_location,job_age,job_posting_description,job_salary_range,job_salary_estimate_type,company,company_total_rating,proportion_reviewers_recommend_company,company_individual_ratings,company_type_size_sector_industry_yearFounded
0,1008762069830,Senior Data Analyst,Remote,25d,We are a leading ecommerce business (electric ...,SGD 50K - SGD 55K (Employer est.),(Employer est.),VORO MOTORS3.8 ★,3.8,57 %,"['Career Opportunities', '3.0', '', '', 'Cultu...","['Company - Private', '51 to 200 Employees', '..."
1,1008799302857,IT Business Analyst (No coding / High Bonus/ W...,Singapore,6d,IT Business Analyst (No coding / High Bonus/ W...,SGD 5K - SGD 9K (Employer est.),(Employer est.),SEARCH INDEX PTE. LTD.3.6 ★,3.6,63 %,"['Career Opportunities', '3.2', 'Comp & Benefi...","['Self-employed', '1 to 50 Employees', None, N..."
2,1008810858685,Business Analyst (Perm Role),Singapore,24h,Provide on-going application support and be in...,SGD 7K - SGD 10K (Employer est.),(Employer est.),BGC GROUP PTE. LTD.3.7 ★,3.7,49 %,"['Career Opportunities', '3.1', 'Comp & Benefi...","['Company - Public', '51 to 200 Employees', 'H..."
3,1008803997038,IT Application Business Analyst,Singapore,3d,Job Description:\n\nScope of Service- The IT A...,SGD 69K - SGD 90K (Glassdoor est.),(Glassdoor est.),Cognizant3.8 ★,3.8,75 %,"['Career Opportunities', '3.7', 'Comp & Benefi...","['Company - Public', '10000+ Employees', 'Info..."
4,1008806044086,Business Analyst - Specialist,Singapore,2d,Employment Type\nPermanent\nClosing Date\n16 A...,NaN,NaN,Telstra3.9 ★,3.9,77 %,"['Career Opportunities', '3.6', 'Comp & Benefi...","['Company - Public', '10000+ Employees', 'Tele..."


In [5]:
df["job_location"].value_counts()

job_location
Singapore                     1096
Remote                           2
Queenstown Estate                2
Jurong West New Town             2
Novena                           2
Toa Payoh New Town               1
Old Kallang Airport Estate       1
Geylang Serai                    1
Serangoon                        1
Name: count, dtype: int64

In [6]:
def simplify_location(location):
    """
    Converts location (string object) into another string object - "Remote" or "Singapore" accordingly
    """
    if str(location).strip() == "Remote": return "Remote"
    else: return "Singapore"
df["job_location"] = df["job_location"].apply(simplify_location)

In [7]:
def clean_age(length_of_time: str) -> str:
    """
    Converts into number of days. If length_of_time > 30d+, we will leave it as is.
    
    Example:
    clean_age("30d+") -> "30 days +"
    clean_age("5d") -> "5 days"
    clean_age("12h") -> "0.5 days"
    """
    
    for i,c in enumerate(length_of_time):
        if c.isalpha():
            number = length_of_time[:i]
            unit = length_of_time[i:]

    if unit == "d": return f"{number} days"
    elif unit == "d+": return f"{number} days +"
    elif unit == "h": return f"{round(int(number)/24,1)} days"
    else:
        print("new data that does not have d,d+,h as units. Need edit code")
        raise Exception

df["job_age"] = df["job_age"].apply(clean_age)

In [8]:
def simplify_salary(salary_range:str) -> int:
    """
    Convert a salary range to an estimate of the salary by taking an average. Then,
    it will return an integer value of the estimate in SGD.

    E.g.
    simplify_salary("SGD 50K - SGD 55K (Employer est.)") -> 52500
    simplify_salary(Nan) -> Nan
    simplify_salary("SGD 7K (Employer est.)") -> 7000
    """
    lower_bound = None
    upper_bound = None

    try:
        for word in salary_range.split(" "):
            if word[0].isnumeric():
                for i,c in enumerate(word):
                    if c.isalpha():
                        number = word[:i]
                #update bound
                if lower_bound == None: lower_bound = int(number)
                else: upper_bound = int(number)
        return (lower_bound + upper_bound)*1000/2
        
    except AttributeError: #To catch Nan values (missing values)
        return salary_range
        
    except TypeError: #To catch values with no upper bound/lower bound
        return int(lower_bound)
        
df["job_salary_range"] = df["job_salary_range"].apply(simplify_salary) #to rename column later as it is not a range

In [9]:
df = df.dropna(subset = ["company"])

def clean_company(name:str) -> str:
    """
    Removes the rating, and keeps the name of the company.
    Code assumes that there is no number in the name of the company.
    
    E.g.
    clean_company("	Telstra3.9 ★") -> "Telstra"
    """
    result = name.strip()
    for i,c in enumerate(result):
        if c.isnumeric():
            return result[:i]

df["company"] = df["company"].apply(clean_company)

In [10]:
"""
To change:
1. company_total_rating into a float (ignore NA values)
2. job_name
3. job_posting_description
"""

'\nTo change:\n1. company_total_rating into a float (ignore NA values)\n2. job_name\n3. job_posting_description\n'

In [11]:
def clean_proportion(proportion:str) -> str:
    try:
        return int(proportion.strip().split(" ")[0])
    except AttributeError: #To catch Nan values (missing values)
        return proportion

df["proportion_reviewers_recommend_company"] = df["proportion_reviewers_recommend_company"].apply(clean_proportion)

In [49]:
#Cleaning individual ratings
def separate_individual_ratings(individual_ratings):
    """
    Function takes the company_individual_ratings variable (Panda Series object) as input and seperates the individual ratings.
    It outputs a dictionary representing the individual ratings.

    E.g.
    individual_ratings = [ "['Career Opportunities', '1.0', 'Comp & Benefits', '1.0', 'Culture & Values', '1.0', 'Senior Management', '1.0', 'Work/Life Balance', '3.0']",
                           "['Career Opportunities', '3.1', 'Comp & Benefits', '3.0', 'Culture & Values', '3.3', 'Senior Management', '3.2', 'Work/Life Balance', '3.6']",
                           "['Career Opportunities', '3.0', '', '', 'Culture & Values', '3.1', 'Senior Management', '3.0', 'Work/Life Balance', '3.5']"
                         ]
                         
    separate_individual_ratings(individual_ratings) -> {"Career Opportunities" : [1.0, 3.1, 3.0],
                                                        "Comp & Benefits" : [1.0, 3.0, Nan],
                                                        "Culture & Values" : [1.0,3.3,3.1],
                                                        "Senior Management" : [1.0, 3.2, 3.0],
                                                        "Work_life_balance" : [3.0, 3.6, 3.5]
                                                       }
    
    """
    categories = ['Career Opportunities','Comp & Benefits','Culture & Values','Senior Management', 'Work/Life Balance']
    result = {e:[] for e in categories}

    def clean(obs):
        #Takes an observeration string and cleans it
        for e in ["[", "]", "'"]: #Remove these characters
            obs = obs.replace(e,"")
        obs = list(filter(lambda e: e != " ",obs.split(",")))#splitting them and remove empty 'substrings'
        obs = [e.strip() for e in obs]#Remove leading and trailing white spaces
        return obs
        
    def not_float(e): #Returns True if string is not a float. False otherwise
        try:
            float(e)
            return False
        except:
            return True

    def insert_na_row_to_result():
        for e in result: result[e].append(None)
            
    for obs in individual_ratings:
        categories_encountered = []
        #Handling Nan obs
        if obs == "[]":
            insert_na_row_to_result()
            continue
        try:
            cleaned_obs = clean(obs)
        except AttributeError: #obs is Nan
            insert_na_row_to_result()
            continue
                
        #Adding individual scores
        for index,element in enumerate(cleaned_obs):
            if not not_float(element): continue
            else:
                category = element
                score = float(cleaned_obs[index + 1])
                result[category].append(score)
                categories_encountered.append(category)

        #Adding na values for ratings not given
        for c in categories:
            if c not in categories_encountered:
                result[c].append(None)
    return result
            
individual_ratings_dictionary = separate_individual_ratings(df["company_individual_ratings"])

for category,values in individual_ratings_dictionary.items():
    df.insert(9,category,values)
df.pop("company_individual_ratings")

In [61]:
#Cleaning company_type_size_sector_industry_yearFounded
def separate_organisation_details(organisation_details):
    """
    This function is similar to separate_individual_ratings function.
    Function takes the company_type_size_sector_industry_yearFounded variable (Panda Series object) as input and seperates the individual ratings.
    It outputs a dictionary representing the individual ratings.

    E.g.
    individual_ratings = [ "['Career Opportunities', '1.0', 'Comp & Benefits', '1.0', 'Culture & Values', '1.0', 'Senior Management', '1.0', 'Work/Life Balance', '3.0']",
                           "['Career Opportunities', '3.1', 'Comp & Benefits', '3.0', 'Culture & Values', '3.3', 'Senior Management', '3.2', 'Work/Life Balance', '3.6']",
                           "['Career Opportunities', '3.0', '', '', 'Culture & Values', '3.1', 'Senior Management', '3.0', 'Work/Life Balance', '3.5']"
                         ]
                         
    separate_individual_ratings(individual_ratings) -> {"Career Opportunities" : [1.0, 3.1, 3.0],
                                                        "Comp & Benefits" : [1.0, 3.0, Nan],
                                                        "Culture & Values" : [1.0,3.3,3.1],
                                                        "Senior Management" : [1.0, 3.2, 3.0],
                                                        "Work_life_balance" : [3.0, 3.6, 3.5]
                                                       }
    
    """

df.iloc[0,15]

"['Company - Private', '51 to 200 Employees', 'Retail & Wholesale', 'Vehicle Dealers', '2017']"

In [ ]:
df = df.rename({"job_location": "location",
           "job_age": "age",
          "job_salary_range": "salary_estimate",
               "job_salary_estimate_type": "salary_estimate_type"}, axis = 1)
df.head()